In [9]:
import numpy as np
import matplotlib.pyplot as plt
import math
import plotly.express as px
import pandas as pd

df = pd.read_csv("alt_fuel_stations.csv", dtype='unicode')
#print(df.columns)





Data Cleaning Process

In [10]:
df_clean = df[df.iloc[:,0] == 'ELEC']

df_clean['Latitude'] = df_clean['Latitude'].astype(float)
df_clean['Longitude'] = df_clean['Longitude'].astype(float)
df_clean['Lat_Log_Coordinates'] = list(zip(df_clean['Latitude'], df_clean['Longitude']))
df_clean['Lat_Log_Coordinates']
df_clean.columns

C:\Users\DDorfman\AppData\Local\Temp\ipykernel_5904\4137543191.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DDorfman\AppData\Local\Temp\ipykernel_5904\4137543191.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DDorfman\AppData\Local\Temp\ipykernel_5904\4137543191.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

Derive the top 15 cities based on number of chargers

In [11]:
top_cities = df_clean['City'].value_counts().head(15).reset_index().set_index('City')
#print(top_cities)
print(np.array(top_cities.index))


[1644  808  628  621  602  585  489  487  452  444  404  377  363  359
  349]


Calculating the Average distance to the nearest charging station for each station

In [12]:
import itertools
import math

top_cities_arr = np.array(top_cities.index)
df_city_view = df_clean[['City','Lat_Log_Coordinates']].dropna().set_index('City').loc[top_cities_arr]


def geo_distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 3958.7564 #20925721
    # Convert all angles to radians
    lat1_r = math.radians(lat1)
    lon1_r = math.radians(lon1)
    lat2_r = math.radians(lat2)
    lon2_r = math.radians(lon2)
    # Calculate the distance
    dp = math.cos(lat1_r) * math.cos(lat2_r) * math.cos(lon1_r - lon2_r) + math.sin(lat1_r) * math.sin(lat2_r)

    if dp > 1:
        dp = 1
    elif dp < -1:
        dp = -1
    angle = math.acos(dp)

    return EARTH_RADIUS * angle


def average_shortest_distance(ls):
    output = ls
    final_list = []
    for i in ls:
        temp_dist = []
        for j in ls:

            temp_dist.append(geo_distance(float(i[0]),
                                          float(i[1]),
                                          float(j[0]),
                                          float(j[1])))

        #index = min(temp_dist)
        final_list.append(temp_dist)

    return final_list

def average_distance(ls):
    #print(grp)
    #list=grp.loc['Lat_Log_Coordinates']
    output = []
    for i,j in itertools.combinations(ls,2 ):  
        
        #dist = sum([i,j])
        dist = geo_distance(i[0],i[1],j[0],j[1])
        
        output.append(dist)
    return np.mean(np.array(output), axis=0)


table = df_city_view
table['helper'] = df_city_view.groupby('City').transform(average_shortest_distance)
table['miles_to_nearest_charger'] = table['helper'].apply(sorted).apply(lambda x: x[1])
table2 = table.groupby('City')['miles_to_nearest_charger'].mean()
table2.reset_index().set_index('City')


    

KeyError: "None of [Int64Index([1644, 808, 628, 621, 602, 585, 489, 487, 452, 444, 404, 377, 363,\n            359, 349],\n           dtype='int64', name='City')] are in the [index]"

Calculating Charging Station Density

In [13]:
#Calculating Density

city_density_sq_mi = {
    'Atlanta': 133.2,
    'Austin': 319.9,
    'Boston': 48.34,
    'Chicago': 227.7,
    'Irvine': 65.6,
    'Kansas City': 314.7,
    'Los Angeles': 469.5,
    'Menlo Park': 9.99,
    'New York': 300.4,
    'Sacramento': 97.7,
    'San Diego': 325.9,
    'San Francisco': 46.9,
    'San Jose': 178.2,
    'Seattle': 84.0,
    'Washington': 61.1
}

top_cities_density = top_cities.merge(pd.Series(city_density_sq_mi, name = 'land_sq_mi'),left_index = True, right_index = True)
top_cities_density['EV Charger Density'] = top_cities_density['count']/top_cities_density['land_sq_mi']
top_cities_density


KeyError: 'count'

Determining Latitude and Longitude for each city

In [14]:

#df_clean.groupby('City').mean().sort_values(ascending = False).head(15)
df_coord = df_clean[df_clean['City'].isin(np.array(top_cities.index))].groupby('City')[['Latitude','Longitude']].mean()
us_cities = top_cities.merge(df_coord, left_index = True, right_index = True)
us_cities
#
#np.array(top_cities.index)

,index,Latitude,Longitude
City,,,


Map of Size of Number of Chargers for the top 15 Cities

In [15]:
import geopandas as gpd

# Load the data
us_cities = us_cities
#us_map = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
us_map = gpd.read_file('../map/cb_2018_us_state_500k.shp')
us_map = us_map[(us_map['NAME'] != 'Alaska') & (us_map['NAME'] != 'Hawaii') & (us_map['NAME'] != 'Puerto Rico')]
#us_map = us_map[(us_map['name'] == 'United States of America') & (us_map['continent'] == 'North America')]
print(us_map.head())

fig,ax = plt.subplots(figsize = (20, 5))


us_map.plot(ax = ax)

ax.set_ylim([25, 50])
ax.set_xlim([ -140,-50])
gdf = gpd.GeoDataFrame(
    us_cities, geometry=gpd.points_from_xy(us_cities.Longitude, us_cities.Latitude))

gdf.plot(ax=ax, color='blue', markersize=us_cities['count'], alpha = 0.5)

"""
#us_map = us_map.explode().reset_index(drop=True)
#us_map = us_map[~((us_map.bounds['minx'] < -120) & (us_map.bounds['maxx'] > 0))]
#us_map = us_map[us_map.STATE_ABBR ]
# Convert the data to a GeoDataFrame
gdf = gpd.GeoDataFrame(
    us_cities, geometry=gpd.points_from_xy(us_cities.Longitude, us_cities.Latitude))

# Plot the data
ax = us_map.plot(color='white', edgecolor='black')
gdf.plot(ax=ax, color='blue', markersize=us_cities['count'], alpha = 0.5)
"""

ModuleNotFoundError: No module named 'geopandas'

Interactive Heat Map of Charging Stations

In [16]:
#scatter_geo
#density_mapbox
fig = px.density_mapbox(df_clean,lat='Latitude',lon='Longitude'
                        , mapbox_style="stamen-terrain"
                        , radius=5)
fig.update_layout(title = 'Map of Charging Stations', title_x=0.5)
fig.show()



